### Linear Regression

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline

In [2]:
spark = SparkSession.builder.appName("Linear Regression").getOrCreate()
data = spark.read.csv("..\Data\Car_Sales.csv", header = True, inferSchema = True)
data.show()

+-------------+------+---------+-------+-------+-----------+------------+----+----------------+
|        Brand| Price|     Body|Mileage|EngineV|Engine Type|Registration|Year|           Model|
+-------------+------+---------+-------+-------+-----------+------------+----+----------------+
|          BMW|  4200|    sedan|    277|      2|     Petrol|         yes|1991|             320|
|Mercedes-Benz|  7900|      van|    427|    2.9|     Diesel|         yes|1999|    Sprinter 212|
|Mercedes-Benz| 13300|    sedan|    358|      5|        Gas|         yes|2003|           S 500|
|         Audi| 23000|crossover|    240|    4.2|     Petrol|         yes|2007|              Q7|
|       Toyota| 18300|crossover|    120|      2|     Petrol|         yes|2011|           Rav 4|
|Mercedes-Benz|199999|crossover|      0|    5.5|     Petrol|         yes|2016|          GLS 63|
|          BMW|  6100|    sedan|    438|      2|        Gas|         yes|1997|             320|
|         Audi| 14200|    vagon|    200|

In [3]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import IntegerType
def OneHotEncoding(df, col_name = ""):
    ###Gather the distinct values 
    distinct_values = list(df.select(col_name).distinct().toPandas()[col_name])
    # for each of the gathered values create a new column
#For Body
    for distinct_value in distinct_values:
        function = udf(lambda item: 1 if item == distinct_value else 0, IntegerType())
        new_column_name = col_name+'_'+distinct_value
        df = df.withColumn(new_column_name, function(col(col_name)))
    
    return df

In [4]:
data.columns

['Brand',
 'Price',
 'Body',
 'Mileage',
 'EngineV',
 'Engine Type',
 'Registration',
 'Year',
 'Model']

In [5]:
#Encodin gBrand
data = OneHotEncoding(data, col_name = "Brand")

In [6]:
#Encoding Engine Type Column
data = OneHotEncoding(data, col_name = "Engine Type")

In [7]:
#Encoding Body and Registration Columns
data = OneHotEncoding(data, col_name = "Body")
data = OneHotEncoding(data, col_name = "Registration")
data.columns

['Brand',
 'Price',
 'Body',
 'Mileage',
 'EngineV',
 'Engine Type',
 'Registration',
 'Year',
 'Model',
 'Brand_Volkswagen',
 'Brand_Mitsubishi',
 'Brand_Audi',
 'Brand_Mercedes-Benz',
 'Brand_Renault',
 'Brand_BMW',
 'Brand_Toyota',
 'Engine Type_Diesel',
 'Engine Type_Other',
 'Engine Type_Gas',
 'Engine Type_Petrol',
 'Body_van',
 'Body_crossover',
 'Body_other',
 'Body_sedan',
 'Body_hatch',
 'Body_vagon',
 'Registration_no',
 'Registration_yes']

In [8]:
data.printSchema()

root
 |-- Brand: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Body: string (nullable = true)
 |-- Mileage: integer (nullable = true)
 |-- EngineV: string (nullable = true)
 |-- Engine Type: string (nullable = true)
 |-- Registration: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Model: string (nullable = true)
 |-- Brand_Volkswagen: integer (nullable = true)
 |-- Brand_Mitsubishi: integer (nullable = true)
 |-- Brand_Audi: integer (nullable = true)
 |-- Brand_Mercedes-Benz: integer (nullable = true)
 |-- Brand_Renault: integer (nullable = true)
 |-- Brand_BMW: integer (nullable = true)
 |-- Brand_Toyota: integer (nullable = true)
 |-- Engine Type_Diesel: integer (nullable = true)
 |-- Engine Type_Other: integer (nullable = true)
 |-- Engine Type_Gas: integer (nullable = true)
 |-- Engine Type_Petrol: integer (nullable = true)
 |-- Body_van: integer (nullable = true)
 |-- Body_crossover: integer (nullable = true)
 |-- Body_other: integer (nulla

In [9]:
data = data.drop( "Engine Type", "Body", "Registration", "Brand")


In [10]:
data.columns

['Price',
 'Mileage',
 'EngineV',
 'Year',
 'Model',
 'Brand_Volkswagen',
 'Brand_Mitsubishi',
 'Brand_Audi',
 'Brand_Mercedes-Benz',
 'Brand_Renault',
 'Brand_BMW',
 'Brand_Toyota',
 'Engine Type_Diesel',
 'Engine Type_Other',
 'Engine Type_Gas',
 'Engine Type_Petrol',
 'Body_van',
 'Body_crossover',
 'Body_other',
 'Body_sedan',
 'Body_hatch',
 'Body_vagon',
 'Registration_no',
 'Registration_yes']

In [11]:
#Checking the number of Columns in the our dataset
len(data.columns)

24

In [12]:
#Converting String Columns into Float
from pyspark.sql.types import DoubleType
data = data.withColumn("Price", data['Price'].cast("double"))
data = data.withColumn("EngineV", data['EngineV'].cast("double"))
data.printSchema()


root
 |-- Price: double (nullable = true)
 |-- Mileage: integer (nullable = true)
 |-- EngineV: double (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Model: string (nullable = true)
 |-- Brand_Volkswagen: integer (nullable = true)
 |-- Brand_Mitsubishi: integer (nullable = true)
 |-- Brand_Audi: integer (nullable = true)
 |-- Brand_Mercedes-Benz: integer (nullable = true)
 |-- Brand_Renault: integer (nullable = true)
 |-- Brand_BMW: integer (nullable = true)
 |-- Brand_Toyota: integer (nullable = true)
 |-- Engine Type_Diesel: integer (nullable = true)
 |-- Engine Type_Other: integer (nullable = true)
 |-- Engine Type_Gas: integer (nullable = true)
 |-- Engine Type_Petrol: integer (nullable = true)
 |-- Body_van: integer (nullable = true)
 |-- Body_crossover: integer (nullable = true)
 |-- Body_other: integer (nullable = true)
 |-- Body_sedan: integer (nullable = true)
 |-- Body_hatch: integer (nullable = true)
 |-- Body_vagon: integer (nullable = true)
 |-- Registration_n

##### Feature Assembling

In [13]:
from pyspark.ml.feature import VectorAssembler

featureassembler = VectorAssembler(inputCols = ['Price', 'Mileage', 'EngineV', 'Year', 'Brand_Volkswagen', 'Brand_Mitsubishi', 'Brand_Audi', 'Brand_Mercedes-Benz', 'Brand_Renault', 'Brand_BMW', 'Brand_Toyota', 'Engine Type_Diesel',
 'Engine Type_Other',
 'Engine Type_Gas',
 'Engine Type_Petrol',
 'Body_van',
 'Body_crossover',
 'Body_other',
 'Body_sedan',
 'Body_hatch',
 'Body_vagon',
 'Registration_no',
 'Registration_yes'], outputCol = 'features')

output = featureassembler.transform(data)

In [14]:
output.columns


['Price',
 'Mileage',
 'EngineV',
 'Year',
 'Model',
 'Brand_Volkswagen',
 'Brand_Mitsubishi',
 'Brand_Audi',
 'Brand_Mercedes-Benz',
 'Brand_Renault',
 'Brand_BMW',
 'Brand_Toyota',
 'Engine Type_Diesel',
 'Engine Type_Other',
 'Engine Type_Gas',
 'Engine Type_Petrol',
 'Body_van',
 'Body_crossover',
 'Body_other',
 'Body_sedan',
 'Body_hatch',
 'Body_vagon',
 'Registration_no',
 'Registration_yes',
 'features']

In [15]:
(train, test) = output.randomSplit([0.7, 0.3], 123445)

In [16]:
lr  = LinearRegression(featuresCol = "features", labelCol = "Price",maxIter = 10, regParam = 0.3, elasticNetParam = 0.8)
lrModel = lr.fit(train)

Py4JJavaError: An error occurred while calling o359.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 15.0 failed 1 times, most recent failure: Lost task 0.0 in stage 15.0 (TID 11) (Abdul-Majeed executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:188)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:108)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:121)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:162)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.evaluate(BatchEvalPythonExec.scala:81)
	at org.apache.spark.sql.execution.python.EvalPythonExec.$anonfun$doExecute$2(EvalPythonExec.scala:130)
	at org.apache.spark.sql.execution.python.EvalPythonExec$$Lambda$3413/282444629.apply(Unknown Source)
	at org.apache.spar

In [ ]:
#Summarize the model over the training set and print out some metrics
trainingSummary = lrModel.summary
print("numIterations: %d" % train.totalIterations)

In [ ]:
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))

In [ ]:
trainingSummary.residuals.show()

In [ ]:
print("RMSE: %F " % trainingSummary.rootMeanSquaredError)

In [ ]:
print("r2: %f" % trainingSummary.r2)

In [ ]:
prediction = lrModel.transform(test)
prediction.show()